In [ ]:
import requests
import os
import json
import base64

def test_sqlite_workflow():
    # API endpoint
    API_URL = "http://localhost:5000/query"

    # Path to your SQLite database file - REPLACE THIS WITH YOUR ACTUAL PATH
    sqlite_path = "../spider_data/sqlite_spider/car_1.sqlite"  # Example path, replace with your actual SQLite file
    database_id = sqlite_path.split("/")[-1].split(".")[0]

    try:
        print(f"Attempting to read SQLite file from: {sqlite_path}")
        # Read SQLite file in binary mode
        with open(sqlite_path, 'rb') as f:
            sqlite_binary = f.read()
        
        file_size_kb = len(sqlite_binary) / 1024
        print(f"Successfully read SQLite file ({file_size_kb:.2f} KB)")
        
        # Encode binary data as Base64 string
        print("Encoding file as Base64...")
        sqlite_base64 = base64.b64encode(sqlite_binary).decode('utf-8')

        # Create connection payload for SQLite
        sqlite_connection_payload = {
            "file": sqlite_base64,
            "dbType": "sqlite"
        }

        

        # Test queries
        test_queries = [
            "How many models does each car maker produce? List maker full name, id and the number.",
        ]

        for query in test_queries:
            print(f"\nTesting query: {query}")
            
            # Prepare request payload
            payload = {
                "query": query,
                "connection_payload": sqlite_connection_payload
            }

            # Make POST request to the endpoint
            print(f"Sending request to {API_URL}...")
            response = requests.post(API_URL, json=payload)

            # Print response
            if response.status_code == 200:
                result = response.json()
                print(f"Generated SQL: {result['sql']}")
            else:
                print(f"Error: {response.status_code}")
                print(f"Response: {response.text}")

    except FileNotFoundError:
        print(f"Error: SQLite file not found at {sqlite_path}")
        print("Please update the 'sqlite_path' variable with the correct path to your SQLite database file.")
    except Exception as e:
        print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    print("SQLite Database Test Script")
    print("==========================")
    print("This script tests the SQL generation workflow using a SQLite database.")
    print("Make sure your Flask app is running at http://localhost:5000\n")
    test_sqlite_workflow() 

SQLite Database Test Script
This script tests the SQL generation workflow using a SQLite database.
Make sure your Flask app is running at http://localhost:5000

Attempting to read SQLite file from: ../spider_data/sqlite_spider/car_1.sqlite
Successfully read SQLite file (64.00 KB)
Encoding file as Base64...

Testing query: How many models does each car maker produce? List maker full name, id and the number.
Sending request to http://localhost:5000/query...
Generated SQL: SELECT 
    cm.FullName, 
    cm.Id AS MakerId,
    COUNT(ml.Model) AS NumberOfModels
FROM 
    car_makers cm
JOIN 
    model_list ml ON cm.Id = ml.Maker
GROUP BY 
    cm.FullName, cm.Id;
